In [1]:
import datalab.bigquery as bq
import seaborn as sns
import pandas as pd
import numpy as np
import shutil
import tensorflow as tf
print tf.__version__
import os

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.7.0


In [2]:
PROJECT = 'go-de-internal'
BUCKET = 'go-de-internal-johanna'
REGION = 'europe-west1'
REPO = "/content/datalab/demo-cases/churn-prediction/lstm"
os.listdir(REPO)

['check.csv',
 'lstm.ipynb',
 'lstm 2.0.ipynb',
 'intermediate.csv',
 'churn_100_customers.csv',
 'churn_100_customers_365_days.csv',
 '.ipynb_checkpoints',
 'data_sets',
 'churn-create-timedependent-data.ipynb',
 'churn-time-dependent-artificial-data.csv',
 'trainer',
 'lstm training.ipynb',
 'churn.ipynb']

In [3]:
# for bash
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['REPO'] = REPO

In [4]:
%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [5]:
%bash
PROJECT_ID=$PROJECT
AUTH_TOKEN=$(gcloud auth print-access-token)
SVC_ACCOUNT=$(curl -X GET -H "Content-Type: application/json" \
    -H "Authorization: Bearer $AUTH_TOKEN" \
    https://ml.googleapis.com/v1/projects/${PROJECT_ID}:getConfig \
    | python -c "import json; import sys; response = json.load(sys.stdin); \
    print response['serviceAccount']")

echo "Authorizing the Cloud ML Service account $SVC_ACCOUNT to access files in $BUCKET"
gsutil -m defacl ch -u $SVC_ACCOUNT:R gs://$BUCKET
gsutil -m acl ch -u $SVC_ACCOUNT:R -r gs://$BUCKET  # error message (if bucket is empty) can be ignored
gsutil -m acl ch -u $SVC_ACCOUNT:W gs://$BUCKET

Authorizing the Cloud ML Service account service-462605511119@cloud-ml.google.com.iam.gserviceaccount.com to access files in go-de-internal-johanna


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   133    0   133    0     0    136      0 --:--:-- --:--:-- --:--:--   136
No changes to gs://go-de-internal-johanna/
No changes to gs://go-de-internal-johanna/churn-prediction/
No changes to gs://go-de-internal-johanna/churn-prediction/lstm/
No changes to gs://go-de-internal-johanna/churn-prediction/lstm/a_b
No changes to gs://go-de-internal-johanna/churn-prediction/lstm/a_b/
No changes to gs://go-de-internal-johanna/churn-prediction/lstm/a_b/a_b.csv
No changes to gs://go-de-internal-johanna/churn-prediction/neural_net/churn-einzelhandel-train.csv
No changes to gs://go-de-internal-johanna/churn-prediction/neural_net/churn_trained/churn_model.hdf5
No changes to gs://go-de-internal-johanna/churn-prediction/neural_net/churn_trained/export/saved_model.pb
No changes to gs://go-de-internal-johanna/churn-prediction/mv_ab_model_ads_b

In [ ]:
!cat trainer/model.py

In [11]:
%bash
head -1 $REPO/labels.csv
head -1 $REPO/training.csv

0
,0,1,2,3,4


In [ ]:
from google.cloud import storage

DATA_BUCKET='go-de-internal-johanna'
FILES = ['training.csv', 'labels.csv']

client = storage.Client(project=PROJECT)
bucket = client.get_bucket(DATA_BUCKET)

for filename in FILES:
    with open(filename, 'wb') as f:
        bucket.blob(filename).download_to_file(f)

In [49]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

training = pd.read_csv('training.csv')
print('1', training)
training = training.iloc[:,1:] #  remove useless first column
labels = pd.read_csv('labels.csv')

labels = labels.iloc[:,1:] #  remove useless first column
print('2', training)
n_users = 10
n_seq = training.shape[0]/n_users
n_features = training.shape[1]
y = np.array(labels)
X = np.array(training).reshape(n_users, n_seq, n_features)
print('3', X, X.shape)
X_train, X_eval, y_train, y_eval = train_test_split(X, y)

('1',       Unnamed: 0  0  1  2  3  4
0              0  0  0  0  0  0
1              1  0  0  0  0  0
2              2  0  0  0  0  0
3              3  0  0  0  0  0
4              4  0  0  0  0  0
5              5  0  0  0  0  0
6              6  0  0  0  0  0
7              7  0  0  0  0  0
8              8  0  0  0  0  0
9              9  0  0  0  0  0
10            10  0  0  0  0  0
11            11  0  0  0  0  0
12            12  0  0  0  0  0
13            13  0  0  0  0  0
14            14  0  0  0  0  0
15            15  0  0  0  0  0
16            16  0  0  0  0  0
17            17  0  0  0  0  0
18            18  0  0  0  0  0
19            19  0  0  0  0  0
20            20  0  0  0  0  0
21            21  0  0  0  0  0
22            22  0  0  0  0  0
23            23  0  0  0  0  0
24            24  0  0  0  0  0
25            25  0  0  0  0  0
26            26  0  0  0  0  0
27            27  0  0  0  0  0
28            28  0  0  0  0  0
29            29  0  0  0  0  0
..

In [45]:
list = [1,2,3,4,56,7,8,9]
array = np.array(list)
print(array)

[ 1  2  3  4 56  7  8  9]


In [29]:
import cPickle as pickle

FILES = ['X_train.pkl', 'X_eval.pkl', 'y_train.pkl', 'y_eval.pkl']
ARRAYS = [X_train, X_eval, y_train, y_eval]
for idx, filename in enumerate(FILES):
    with open("pickels/"+filename, 'wb') as f:
        pickle.dump(ARRAYS[idx], f)

## Compile Model:
Note the Stacked LSTMs...

In [31]:
!pip install keras

  Using cached Keras-2.1.5-py2.py3-none-any.whl
You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [32]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM

def model_fn(
    n_seq, n_features, 
    lstm1_nodes=10, lstm2_nodes=10, mlp1_nodes=10
):
    model = Sequential()
    model.add(LSTM(lstm1_nodes, input_shape=(n_seq, n_features), return_sequences=True))
    model.add(LSTM(lstm2_nodes))
    model.add(Dense(mlp1_nodes, activation='relu', name='last_layer'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(
        loss='binary_crossentropy',
        optimizer='rmsprop',
        metrics=['accuracy']
    )
    
    return model

Using TensorFlow backend.


## Train Model Locally:

In [38]:
BATCH_SIZE = 32

model = model_fn(X.shape[1], X.shape[2])
#print(X_eval) 
print(y_train)
model.fit(
    X_train, y_train,
    batch_size=BATCH_SIZE, epochs=3,
    validation_data=(X_eval, X_eval)
)

[]


ValueError: Error when checking target: expected dense_6 to have shape (1,) but got array with shape (0,)

In [ ]:
%bash
rm -rf vehicles.tar.gz lstm_trained
export PYTHONPATH=${PYTHONPATH}:${REPO}
python -m trainer.task \
   --train_data_paths="${REPO}/training.csv" \
   --eval_data_paths=${REPO}/.csv  \
   --output_dir=${REPO}/vehicles_trained \
   --num_epochs=10 \
   --job-dir=./tmp

In [ ]:
!ls $REPO/vehicles_trained/export/Servo/

In [ ]:
%writefile ./test.json
{"ACCELERATION": 0.3424933,"PITCH": 0.4929889823,"ROLL": 0.999398298932,"YAW": 0.103000023}

In [ ]:
%bash
model_dir=$(ls ${REPO}/vehicles_trained/export/Servo/)
gcloud ml-engine local predict \
    --model-dir=${REPO}/vehilces_trained/export/Servo/${model_dir} \
    --json-instances=./test.json

In [ ]:
%bash
rm -rf vehicles.tar.gz vehicles_trained
gcloud ml-engine local train \
   --module-name=trainer.task \
   --package-path=${REPO}/econsumption/trainer \
   -- \
   --train_data_paths=${REPO}/vehicles-train.csv \
   --eval_data_paths=${REPO}/vehicles-valid.csv  \
   --num_epochs=10 \
   --output_dir=${REPO}/vehicles_trained 

In [ ]:
from google.datalab.ml import TensorBoard
TensorBoard().start('{}/vehicles_trained'.format(REPO))

In [ ]:
for pid in TensorBoard.list()['pid']:
  TensorBoard().stop(pid)
  print 'Stopped TensorBoard with pid {}'.format(pid)

In [ ]:
!ls $REPO/vehicles_trained

In [ ]:
%bash
model_dir=$(ls ${REPO}/vehicles_trained/export/Servo)
gcloud ml-engine local predict \
    --model-dir=${REPO}/vehilces_trained/export/Servo/${model_dir} \
    --json-instances=./test.json

In [ ]:
%bash
echo $BUCKET
gsutil -m rm -rf gs://${BUCKET}/vehilce-health-status/neural-net
gsutil -m cp ${REPO}/*.csv gs://${BUCKET}/vehilce-health-status/neural-net/

In [ ]:
%%bash
OUTDIR=gs://${BUCKET}/vehilce-health-status/neural-net/vehicles_trained
JOBNAME=vehicles_health_demo_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=${REPO}/econsumption/trainer \
   --job-dir=$OUTDIR \
   --staging-bucket=gs://$BUCKET \
   --scale-tier=BASIC \
   --runtime-version=1.0 \
   -- \
   --train_data_paths="gs://${BUCKET}/vehilce-health-status/neural-net/vehicles-train*" \
   --eval_data_paths="gs://${BUCKET}/vehilce-health-status/neural-net/vehicles-valid*"  \
   --output_dir=$OUTDIR \
   --num_epochs=100